In [2]:
import sys
sys.path.append("../metaheuristics") 
%load_ext autoreload
%autoreload 1
%aimport algorithme_genetique

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from algorithme_genetique import*

In [4]:
vrptw = load_solomon('simple.csv', nb_cust=10, vehicle_speed=100)
instance =init_vrpga(vrptw)
best_solutions={}

In [81]:
for i in range(40):
    instance.evolution()
print(instance.best_solution,instance.fitness(instance.best_solution))

[0, 5, 3, 0, 6, 10, 4, 0, 8, 0, 7, 9, 1, 0, 2, 0] -5620.84314940416


In [82]:
for i in range(10):
    for i in range(400):
        instance.evolution()
    print(instance.best_solution,instance.fitness(instance.best_solution))
    best_solutions[str(instance.best_solution)]=instance.fitness(instance.best_solution)

[0, 5, 7, 3, 6, 4, 10, 1, 0, 8, 9, 2, 0] -2411.0398677072353
[0, 5, 10, 6, 4, 1, 0, 3, 7, 8, 9, 2, 0] -2469.901798976494
[0, 5, 7, 3, 10, 1, 0, 8, 6, 4, 9, 2, 0] -2425.6667036401786
[0, 5, 2, 0, 7, 8, 9, 10, 6, 4, 0, 3, 1, 0] -3476.5467041758675
[0, 7, 8, 9, 0, 3, 6, 4, 2, 10, 1, 0, 5, 0] -3425.4882803104597
[0, 7, 8, 2, 0, 3, 6, 4, 9, 10, 1, 0, 5, 0] -3448.674311569822
[0, 7, 8, 9, 0, 3, 6, 4, 2, 0, 5, 10, 1, 0] -3408.1942041989255
[0, 7, 5, 10, 6, 0, 8, 4, 9, 2, 0, 3, 1, 0] -3545.15106935875
[0, 3, 0, 10, 2, 0, 8, 4, 1, 0, 7, 0, 5, 9, 6, 0] -5611.996581961093
[0, 2, 0, 5, 7, 8, 0, 3, 10, 6, 4, 9, 1, 0] -3477.4161590645153


In [76]:
best_solutions

{'[0, 5, 8, 7, 6, 4, 0, 3, 9, 2, 10, 1, 0]': -2507.0156371393223,
 '[0, 7, 8, 6, 4, 10, 9, 0, 3, 5, 1, 2, 0]': -2493.565753396381,
 '[0, 5, 8, 1, 0, 7, 9, 2, 0, 3, 0, 10, 6, 4, 0]': -4542.028143190721,
 '[0, 5, 10, 3, 6, 4, 0, 2, 1, 0, 9, 0, 8, 7, 0]': -4449.921700970464,
 '[0, 5, 10, 3, 4, 6, 1, 2, 0, 7, 8, 9, 0]': -2383.428459060622}

In [78]:
best_key = max(best_solutions, key=best_solutions.get)
print(best_key)

[0, 5, 10, 3, 4, 6, 1, 2, 0, 7, 8, 9, 0]


Best records:

In [83]:
[0, 5, 3, 7, 8, 9, 10, 6, 4, 2, 1, 0]

[0, 5, 3, 7, 8, 9, 10, 6, 4, 2, 1, 0]